In [102]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import Row, Column
import datetime

In [6]:
spark = SparkSession.builder.appName("lab_2_spark").getOrCreate()
sc = spark.sparkContext

Data frame creation

In [9]:
l = [(datetime.date(2018,1,3), 'Ankit',25, "Girl"),
     (datetime.date(2018,2,3), 'Jalfaizy',22, "Girl"),
     (datetime.date(2018,1,5), 'saurabh',20, "Girl"),
     (datetime.date(2018,1,12), 'Bala',26, "Girl"),
     (datetime.date(2018,7,9), 'Jules',19, "Girl"),
     (datetime.date(2018,3,18), 'Arild',43, "Boy"),
     (datetime.date(2018,1,5), 'sarah',20, "Boy"),
     (datetime.date(2018,8,12), 'Boly',33, "Boy"),
     (datetime.date(2018,4,6), 'Anita',35, "Boy"),
     (datetime.date(2018,12,6), 'Jules',22, "Boy"),
     (datetime.date(2018,7,24), 'Soul',20, "Girl"),
     (datetime.date(2018,6,17), 'Gral',54, "Girl"),
     (datetime.date(2018,9,7), 'Apoh',18, "Girl"),
     (datetime.date(2018,10,4), 'Dony',32, "Girl"),
     (datetime.date(2018,2,5), 'Tanoh',31, "Girl"),
     (datetime.date(2018,11,12), 'Issouf',27, "Boy"),
     (datetime.date(2018,10,3), 'Bilé',29, "Boy"),
     (datetime.date(2018,5,3), 'Gagnon',20, "Boy"),
     (datetime.date(2018,3,5), 'Papiss',28, "Boy"),
     (datetime.date(2018,2,12), 'Kravitz',34, "Boy"),
     (datetime.date(2018,5,9), 'Mouli',35, "Girl"),
     (datetime.date(2018,8,3), 'Jacques',27, "Girl"),
     (datetime.date(2018,12,5), 'soum',22, "Girl"),
     (datetime.date(2018,4,12), 'MBra',36, "Girl")]

rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2]), sex=x[3]))
schemaPeople = spark.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name,sex
0,25,2018-01-03,Ankit,Girl
1,22,2018-02-03,Jalfaizy,Girl
2,20,2018-01-05,saurabh,Girl
3,26,2018-01-12,Bala,Girl
4,19,2018-07-09,Jules,Girl
5,43,2018-03-18,Arild,Boy
6,20,2018-01-05,sarah,Boy
7,33,2018-08-12,Boly,Boy
8,35,2018-04-06,Anita,Boy
9,22,2018-12-06,Jules,Boy


# Question 1

In [26]:
schemaPeople.printSchema()

root
 |-- age: long (nullable = true)
 |-- date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)



In [24]:
# Boys an' Girls
schemaPeople.groupBy("sex").count().show(2, False)

+----+-----+
|sex |count|
+----+-----+
|Boy |10   |
|Girl|14   |
+----+-----+



# Question 2

In [105]:
resum = schemaPeople.groupBy("sex").agg(f.mean("age"),f.min("age"),f.max("age"))
resum.show()

+----+------------------+--------+--------+
| sex|          avg(age)|min(age)|max(age)|
+----+------------------+--------+--------+
| Boy|              29.1|      20|      43|
|Girl|27.642857142857142|      18|      54|
+----+------------------+--------+--------+



# Question 3

In [110]:
medB = schemaPeople.filter(schemaPeople.sex=="Boy").approxQuantile("age",[0.5],0.0)
medG = schemaPeople.filter(schemaPeople.sex=="Girl").approxQuantile("age",[0.5],0.0)
#resum.withColumn("median", Column(medB,medG))

# Question 4

In [117]:
dd = schemaPeople.select("*",f.lit(datetime.date.today()).alias("date_max"))
dd = dd.select("*", f.datediff('date_max', 'date').alias('lndays'))
dd.show(5)

+---+----------+--------+----+----------+------+
|age|      date|    name| sex|  date_max|lndays|
+---+----------+--------+----+----------+------+
| 25|2018-01-03|   Ankit|Girl|2019-10-30|   665|
| 22|2018-02-03|Jalfaizy|Girl|2019-10-30|   634|
| 20|2018-01-05| saurabh|Girl|2019-10-30|   663|
| 26|2018-01-12|    Bala|Girl|2019-10-30|   656|
| 19|2018-07-09|   Jules|Girl|2019-10-30|   478|
+---+----------+--------+----+----------+------+
only showing top 5 rows



# Question 5

In [120]:
dd.select("*", f.datediff('date_max', 'date')).filter(dd.lndays<400).show(5)

+---+----------+------+----+----------+------+------------------------+
|age|      date|  name| sex|  date_max|lndays|datediff(date_max, date)|
+---+----------+------+----+----------+------+------------------------+
| 22|2018-12-06| Jules| Boy|2019-10-30|   328|                     328|
| 32|2018-10-04|  Dony|Girl|2019-10-30|   391|                     391|
| 27|2018-11-12|Issouf| Boy|2019-10-30|   352|                     352|
| 29|2018-10-03|  Bilé| Boy|2019-10-30|   392|                     392|
| 22|2018-12-05|  soum|Girl|2019-10-30|   329|                     329|
+---+----------+------+----+----------+------+------------------------+

